---   
# HW3 - Transfer learning

#### Due October 30, 2019

In this assignment you will learn about transfer learning. This technique is perhaps one of the most important techniques for industry. When a problem you want to solve does not have enough data, we use a different (larger) dataset to learn representations which can help us solve our task using the smaller task.

The general steps to transfer learning are as follows:

1. Find a huge dataset with similar characteristics to the problem you are interested in.
2. Choose a model powerful enough to extract meaningful representations from the huge dataset.
3. Train this model on the huge dataset.
4. Use this model to train on the smaller dataset.


### This homework has the following sections:
1. Question 1: MNIST fine-tuning (Parts A, B, C, D).
2. Question 2: Pretrain on Wikitext2 (Part A, B, C, D)
3. Question 3: Finetune on MNLI (Part A, B, C, D)
4. Question 4: Finetune using pretrained BERT (Part A, B, C)

---   
## Question 1 (MNIST transfer learning)
To grasp the high-level approach to transfer learning, let's first do a simple example using computer vision. 

The torchvision library has pretrained models (resnets, vggnets, etc) on the Imagenet dataset. Imagenet is a dataset
with 1.3 million images covering over 1000 classes of objects. When you use one of these models, the weights of the model initialize
with the weights saved from training on imagenet.

In this task we will:
1. Choose a pretrained model.
2. Freeze the model so that the weights don't change.
3. Fine-tune on a few labels of MNIST.   

#### Choose a model
Here we pick any of the models from torchvision

In [0]:
import torch
import torchvision.models as models

class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# init the pretrained feature extractor
pretrained_resnet18 = models.resnet18(pretrained=True)

# we don't want the built in last layer, we're going to modify it ourselves
pretrained_resnet18.fc = Identity()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 26.8MB/s]


#### Freeze the model
Here we freeze the weights of the model. Freezing means the gradients will not backpropagate
into these weights.

By doing this you can think about the model as a feature extractor. This feature extractor outputs
a **representation** of an input. This representation is a matrix that encodes information about the input.

In [0]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True
        
freeze_model(pretrained_resnet18)

#### Init target dataset
Here we define the dataset we are actually interested in.

In [0]:
import os
from torchvision import transforms
from torchvision.datasets import  MNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

#  train/val  split
mnist_dataset = MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(mnist_dataset, [55000, 5000])

mnist_train = DataLoader(mnist_train, batch_size=32)
mnist_val = DataLoader(mnist_val, batch_size=32)

# test split
mnist_test = DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)

### Part A (init fine-tune model)
decide what model to use for fine-tuning

In [0]:
def init_fine_tune_model():
    
    # YOUR CODE HERE
    
    return fine_tune_model

### Part B (Fine-tune (Frozen))

The actual problem we care about solving likely has a different number of classes or is a different task altogether. Fine-tuning is the process of using the extracted representations (features) to solve this downstream task  (the task you're interested in).

To illustrate this, we'll use our pretrained model (on Imagenet), to solve the MNIST classification task.

There are two types of finetuning. 

#### 1. Frozen feature_extractor
In the first type we pretrain with the FROZEN feature_extractor and NEVER unfreeze it during finetuning.


#### 2. Unfrozen feature_extractor
In the second, we finetune with a FROZEN feature_extractor for a few epochs, then unfreeze the feature extractor and finish training.


In this part we will use the first version

In [0]:
import torch.optim as optim

def FROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, mnist_train, mnist_val):
    """
    model is a feature extractor (resnet).
    Create a new model which uses those features to finetune on MNIST
    
    return the fine_tune model
    """     
    
    # INSERT YOUR CODE: (train the fine_tune model using features extracted by feature_extractor)
    
    pass

### Part C (compute test accuracy)
Compute the test accuracy of fine-tuned model on MNIST

In [0]:
def calculate_mnist_test_accuracy(feature_extractor, fine_tune_model, mnist_test):
    
    test_accuracy = 0 # YOUR CODE HERE...
    
    return test_accuracy

### Grade!
Let's see how you did

In [0]:
def grade_mnist_frozen():
    
    # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    FROZEN_fine_tune_mnist(pretrained_resnet18, fine_tune_model, mnist_train, mnist_val)
    
    # calculate test accuracy
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet18, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy
    
frozen_test_accuracy = grade_mnist_frozen()

### Part D (Fine-tune Unfrozen)
Now we'll learn how to train using the "unfrozen" approach.

In this approach we'll:
1. keep the feature_extract frozen for a few epochs (10)
2. Unfreeze it.
3. Finish training

In [0]:
def UNFROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, mnist_train, mnist_val):
    """
    model is a feature extractor (resnet).
    Create a new model which uses those features to finetune on MNIST
    
    return the fine_tune model
    """     
    
    # INSERT YOUR CODE:
    # keep frozen for 10 epochs
    # ... train
    # unfreeze
    # train for rest of the time
    
    pass

### Grade UNFROZEN
Let's see if there's a difference in accuracy!

In [0]:
def grade_mnist_unfrozen():
    
    # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    FROZEN_fine_tune_mnist(pretrained_resnet50, fine_tune_model, mnist_train, mnist_val)
    
    # calculate test accuracy
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet50, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy
    
unfrozen_test_accuracy = grade_mnist_unfrozen()

In [0]:
assert unfrozen_test_accuracy > frozen_test_accuracy, 'the unfrozen model should be better'

--- 
# Question 2 (train a model on Wikitext-2)

Here we'll apply what we just learned to NLP. In this section we'll make our own feature extractor and pretrain it on Wikitext-2.

The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike License.

#### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

In [1]:
!pip install jsonlines

In [0]:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import os
link = "https://drive.google.com/open?id=1517GeMxCd-vjRyVw-_3w_2MhOre0I0T1"
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
local_download_path = os.path.expanduser('~/data')
try:
    os.makedirs(local_download_path)
except: pass
fname = os.path.join(local_download_path, "wiki-news-300d-1M.vec")
f_ = drive.CreateFile({'id': id})
f_.GetContentFile(fname)
from google.colab import drive
drive.mount('/content/gdrive')

1517GeMxCd-vjRyVw-_3w_2MhOre0I0T1
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import pickle
import torch
import torchvision.models as models
import os
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import os
import json
import jsonlines
import numpy as np
from collections import defaultdict
from torch import nn
import numpy as np
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    embedding_size = 300
    max_vocab_size = 35000
    embedding_dict = {} #np.random.randn(max_vocab_size+2, embedding_size)
    all_train_tokens = []
    i = 0
    
    for line in fin:
        tokens = line.rstrip().split(' ')
        all_train_tokens.append(tokens[0])
        embedding_dict[tokens[0]] = list(map(float, tokens[1:]))
        i += 1
        if i == max_vocab_size:
            break
            
    return embedding_dict, all_train_tokens
  
# download the vectors yourself
fasttext_embedding_dict, all_fasttext_tokens = load_vectors(fname)
  
class LMDataset(Dataset):
    def __init__(self, list_of_token_lists):
        self.input_tensors = []
        self.target_tensors = []

        for sample in list_of_token_lists:
            self.input_tensors.append(torch.tensor([sample[:-1]], dtype=torch.long))
            self.target_tensors.append(torch.tensor([sample[1:]], dtype=torch.long))

    def __len__(self):
        return len(self.input_tensors)

    def __getitem__(self, idx):
        return (self.input_tensors[idx], self.target_tensors[idx])


def tokenize_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for l in tqdm(dataset):
            l = ['<bos>'] + l + ['<eos>']
            encoded_l = dictionary.encode_token_seq(l)
            _current_dictified.append(encoded_l)
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def tokenize_mnli_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for s1, s2 in tqdm(dataset):
            s1 = ['<bos>'] + s1 + ['<eos>']
            s2 = ['<bos>'] + s2 + ['<eos>']
            encoded_s1 = dictionary.encode_token_seq(s1)            
            encoded_s2 = dictionary.encode_token_seq(s2)
            _current_dictified.append([encoded_s1, encoded_s2])
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = max([t.size(-1) for t in list_of_tensors])
    padded_list = []
    for t in list_of_tensors:
        padded_tensor = torch.cat(
            [t, torch.tensor([[pad_token] * (max_length - t.size(-1))], dtype=torch.long)], dim=-1)
        padded_list.append(padded_tensor)

    padded_tensor = torch.cat(padded_list, dim=0)
    return padded_tensor


def pad_collate_fn(pad_idx, batch):
    input_list = [s[0] for s in batch]
    target_list = [s[1] for s in batch]
    input_tensor = pad_list_of_tensors(input_list, pad_idx)
    target_tensor = pad_list_of_tensors(target_list, pad_idx)
    return input_tensor, target_tensor


def load_wikitext(data_dir):
    import subprocess
    filename = os.path.join(data_dir, 'wikitext2-sentencized.json')
    if not os.path.exists(filename):
        os.makedirs(data_dir, exist_ok=True)
        url = "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json"
        args = ['wget', '-O', filename, url]
        subprocess.call(args)
    raw_datasets = json.load(open(filename, 'r'))
    for name in raw_datasets:
        raw_datasets[name] = [x.split() for x in raw_datasets[name]]

    if os.path.exists(os.path.join(data_dir, 'vocab.pkl')):
        vocab = pickle.load(open(os.path.join(data_dir, 'vocab.pkl'), 'rb'))
    else:
        vocab = Dictionary(raw_datasets, include_valid=False)
        pickle.dump(vocab, open(os.path.join(data_dir, 'vocab.pkl'), 'wb'))

    tokenized_datasets = tokenize_dataset(raw_datasets, vocab)
    datasets = {name: LMDataset(ds) for name, ds in tokenized_datasets.items()}
    print("Vocab size: %d" % (len(vocab)))
    return raw_datasets, datasets, vocab


class Dictionary(object):
    def __init__(self, datasets, include_valid=False):
        self.tokens = []
        self.ids = {}
        self.counts = {}
        
        # add special tokens
        self.add_token('<bos>')
        self.add_token('<eos>')
        self.add_token('<pad>')
        self.add_token('<unk>')
        
        for line in tqdm(datasets['train']):
            for w in line:
                self.add_token(w)
                    
        if include_valid is True:
            for line in tqdm(datasets['valid']):
                for w in line:
                    self.add_token(w)
                            
    def add_token(self, w):
        if w not in self.tokens:
            self.tokens.append(w)
            _w_id = len(self.tokens) - 1
            self.ids[w] = _w_id
            self.counts[w] = 1
        else:
            self.counts[w] += 1

    def get_id(self, w):
        return self.ids[w]
    
    def get_token(self, idx):
        return self.tokens[idx]
    
    def decode_idx_seq(self, l):
        return [self.tokens[i] for i in l]
    
    def encode_token_seq(self, l):
        return [self.ids[i] if i in self.ids else self.ids['<unk>'] for i in l]
    
    def __len__(self):
        return len(self.tokens)
 
raw_datasets, datasets, vocab = load_wikitext(os.getcwd())

data_loaders = {name: DataLoader(datasets[name], batch_size=32, shuffle=True,
                                     collate_fn=lambda x: pad_collate_fn(vocab.get_id('<pad>'), x))
                    for name in datasets}
wk2_train_dataloader = data_loaders['train']
wk2_val_dataloader = data_loaders['valid']
wk2_test_dataloader = data_loaders['test']

100%|██████████| 9708/9708 [00:00<00:00, 138380.90it/s]


Vocab size: 33178


In [9]:
print('WIKITEXT 2 dataset statistics')
print(f'training samples:{len(wk2_train_dataloader)}')
print(f'val samples:{len(wk2_val_dataloader)}')
print(f'test samples:{len(wk2_test_dataloader)}')

WIKITEXT 2 dataset statistics
training samples:2447
val samples:265
test samples:304


In [0]:
def init_wikitext_dataset():
    """
    Fill in the details
    """
    return wk2_train_dataloader, wk2_val_dataloader, wk2_test_dataloader

#### Part B   
Here we design our own feature extractor. In MNIST that was a resnet because we were dealing with images. Now we need to pick a model that can model sequences better. Design an RNN-based model here.

In [6]:
# Get the embedding weight matrix from fasttext
def get_weight_m(emb_dim, vocab_dict):
    weight_matrix = np.zeros([len(vocab_dict), emb_dim])
    count = 0

    for i in range(len(vocab_dict)):
        word = vocab_dict.get_token(i)
        if word in fasttext_embedding_dict:
            weight_matrix[i, :] = np.array(fasttext_embedding_dict[word])
        elif word == '<pad>':
            weight_matrix[i, :] = np.zeros(emb_dim)
            count += 1
        elif word == '<unk>':
            weight_matrix[i, :] = np.random.normal(scale = 0.6, size = emb_dim)
            count += 1
        else:
            weight_matrix[i, :] = np.random.normal(scale = 0.6, size = emb_dim)
            count += 1
    print(count)
            
    return torch.from_numpy(weight_matrix).type(torch.FloatTensor)

weight_matrix = get_weight_m(300, vocab)

10473


In [0]:
from torch.nn import Linear, functional
import torch.optim as optim
class LSTMModel(nn.Module):
    """
    This model combines embedding, rnn and projection layer into a single model
    """
    def __init__(self, options):
        super().__init__()
        
        # create each LM part here 
        self.lookup = nn.Embedding.from_pretrained(options["embedding_weight"], freeze=False, padding_idx = options['padding_idx'])
        self.lstm = nn.LSTM(options['input_size'], options['hidden_size'], options['num_layers'], 
                            dropout=options['lstm_dropout'], batch_first=True, bias = options['bias'],
                           bidirectional = options['bid'])
        self.projection = nn.Linear(options['hidden_size'], options['num_embeddings'])
        
    def forward(self, encoded_input_sequence):
        """
        Forward method process the input from token ids to logits
        """
        
        embeddings = self.lookup(encoded_input_sequence)
        lstm_outputs = self.lstm(embeddings) # each batch initialize hidden layers to 0 
        # project of outputs 
        # rnn_outputs: tupple with second element being last hidden state. 
        logits = self.projection(lstm_outputs[0])
        
        return logits

num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

embedding_size = 300
hidden_size = 300
num_layers = 2
lstm_dropout = 0.1
vocab_size = len(vocab)

options = {
    'num_embeddings': vocab_size,
    "embedding_weight": weight_matrix,
    'embedding_dim': embedding_size,
    'padding_idx': vocab.get_id('<pad>'),
    'input_size': embedding_size,
    'hidden_size': hidden_size,
    'num_layers': num_layers,
    'lstm_dropout': lstm_dropout,
    'bias': True,
    'bid': False 
}

In [0]:
def init_feature_extractor():
        
    model = LSTMModel(options).to(current_device)

    # same as previous nn based 
    criterion = nn.CrossEntropyLoss(ignore_index=vocab.get_id('<pad>'))

    model_parameters = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.Adam(model_parameters, lr=0.001)
    
    feature_extractor = {"model": model,
                        "criterion": criterion,
                        "model_parameters": model_parameters,
                        "optimizer": optimizer}
    
    return feature_extractor

#### Part C
Pretrain the feature extractor

In [0]:
def fit_feature_extractor(feature_extractor, wk2_train_dataloader, wk2_val_dataloader):
    
    model = feature_extractor["model"]
    criterion = feature_extractor["criterion"]
    optimizer = feature_extractor["optimizer"]
    
    plot_cache = []
    min_val_loss = 20 
    epoch_num = 5
    for epoch_number in range(epoch_num):
        avg_loss=0
        model.train()
        train_log_cache = []
        for i, (inp, target) in enumerate(wk2_train_dataloader):
            optimizer.zero_grad()
            inp = inp.to(current_device)
            target = target.to(current_device)
            logits = model(inp)
            loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))

            loss.backward()
            optimizer.step()

            train_log_cache.append(loss.item())

            if i % 100 == 0:
                avg_loss = sum(train_log_cache)/len(train_log_cache)
                print('Step {} avg train loss = {:.{prec}f}'.format(i, avg_loss, prec=4))
                train_log_cache = []

        #do valid
        valid_losses = []
        model.eval()
        with torch.no_grad():
            for i, (inp, target) in enumerate(wk2_val_dataloader):
                inp = inp.to(current_device)
                target = target.to(current_device)
                logits = model(inp)

                loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
                valid_losses.append(loss.item())
            avg_val_loss = sum(valid_losses) / len(valid_losses)
            print('Validation loss after {} epoch = {:.{prec}f}'.format(epoch_number, avg_val_loss, prec=4))
            best = avg_val_loss < min_val_loss
            if best:
                min_val_loss = avg_val_loss
                best_model = model
                print("update the best to:")
                print(best_model)
                print("current validation loss is:")
                print(min_val_loss)

        plot_cache.append((avg_loss, avg_val_loss))

    print('Saving best model...')
    #!rm gdrive/My\ Drive/best_300_300_LSTM.pt
    model_save_name = 'best_300_300_LSTM.pt'
    path = f"/content/gdrive/My Drive/{model_save_name}" 
    torch.save({'options': options,
                'loss_cache': plot_cache,
                'model_dict': best_model.state_dict()}, path)
    
    return plot_cache



In [54]:
# Do a test run to see if save and load works
feature_extractor = init_feature_extractor()
fit_feature_extractor(feature_extractor, wk2_train_dataloader, wk2_val_dataloader)

Step 0 avg train loss = 10.4111
Validation loss after 0 epoch = 7.5914
update the best to:
LSTMModel(
  (lookup): Embedding(33178, 300, padding_idx=2)
  (lstm): LSTM(300, 300, num_layers=2, batch_first=True, dropout=0.1)
  (projection): Linear(in_features=300, out_features=33178, bias=True)
)
current validation loss is:
7.59142251809438
Saving best model...


[(9.351080937819047, 7.59142251809438)]

In [0]:
# how to load to a new model
model_save_name = 'best_300_300_LSTM.pt'
path = f"/content/gdrive/My Drive/{model_save_name}" 
original_model = LSTMModel(options).to(current_device)
original_model.load_state_dict(torch.load(path)['model_dict'])
class new_model(nn.Module):
            def __init__(self):
                super().__init__()
                self.features = nn.Sequential(
                    # stop before fc
                    *list(original_model.children())[:-1]
                )
            def forward(self, x):
                x = self.features(x)
                return x

In [73]:
very_new_model = new_model()
for i, (inp, target) in enumerate(wk2_val_dataloader):
    inp = inp.to(current_device)
    print(inp.size())
    target = target.to(current_device)
    logits = very_new_model(inp)
    print(logits[0].size())
    if i > -1:
        break

torch.Size([32, 41])
torch.Size([32, 41, 300])


#### Part D
Calculate the test perplexity on wikitext2. Feel free to recycle code from previous assignments from this class. 

In [0]:
def calculate_wiki2_test_perplexity(feature_extractor, wk2_test_dataloader):
    
    model = feature_extractor["model"]
    criterion = feature_extractor["criterion"]
    optimizer = feature_extractor["optimizer"]
    
    # Load best model and use it on the test set
    
    path = "/content/gdrive/My Drive/best_300_300_LSTM.pt"
    model.load_state_dict(torch.load(path)['model_dict'])
    
    test_losses = []
    model.eval()
    with torch.no_grad():
        for i, (inp, target) in enumerate(wk2_test_dataloader):
            inp = inp.to(current_device)
            target = target.to(current_device)
            logits = model(inp)

            loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
            test_losses.append(loss.item())
        avg_test_loss = np.sum(test_losses) / len(test_losses)

    test_ppl = np.exp(avg_test_loss) # 2**(avg_test_loss/np.log(2))
    
    
    return test_ppl

In [101]:
!cd gdrive/My\ Drive
!ls gdrive/My\ Drive
!rm path
!ls gdrive/My\ Drive

 best_300_300_LSTM.pt  'Colab Notebooks'  'Inference and Representation'
 Capstone	        HW2.pdf		   result_ann_resample_8000
rm: cannot remove 'path': No such file or directory
 best_300_300_LSTM.pt  'Colab Notebooks'  'Inference and Representation'
 Capstone	        HW2.pdf		   result_ann_resample_8000


#### Let's grade your results!
(don't touch this part)

In [11]:
def grade_wikitext2():
    # load data
    wk2_train_dataloader, wk2_val_dataloader, wk2_test_dataloader = init_wikitext_dataset()

    # load feature extractor
    feature_extractor = init_feature_extractor()

    # pretrain using the feature extractor
    fit_feature_extractor(feature_extractor, wk2_train_dataloader, wk2_val_dataloader)

    # check test accuracy
    test_ppl = calculate_wiki2_test_perplexity(feature_extractor, wk2_test_dataloader)

    # the real threshold will be released by Oct 11 
    print(f"Current Test Perplexity is {test_ppl}")
    assert test_ppl < 200, 'ummm... your perplexity is too high...'
    
grade_wikitext2()

Step 0 avg train loss = 10.4160
Step 100 avg train loss = 7.6314
Step 200 avg train loss = 7.1736
Step 300 avg train loss = 7.0669
Step 400 avg train loss = 6.8887
Step 500 avg train loss = 6.6684
Step 600 avg train loss = 6.5581
Step 700 avg train loss = 6.4509
Step 800 avg train loss = 6.3568
Step 900 avg train loss = 6.3117
Step 1000 avg train loss = 6.2383
Step 1100 avg train loss = 6.1726
Step 1200 avg train loss = 6.1260
Step 1300 avg train loss = 6.0507
Step 1400 avg train loss = 6.0193
Step 1500 avg train loss = 5.9996
Step 1600 avg train loss = 5.9510
Step 1700 avg train loss = 5.9284
Step 1800 avg train loss = 5.8862
Step 1900 avg train loss = 5.8435
Step 2000 avg train loss = 5.8239
Step 2100 avg train loss = 5.7943
Step 2200 avg train loss = 5.7566
Step 2300 avg train loss = 5.7314
Step 2400 avg train loss = 5.6717
Validation loss after 0 epoch = 5.5463
update the best to:
LSTMModel(
  (lookup): Embedding(33178, 300, padding_idx=2)
  (lstm): LSTM(300, 300, num_layers=2, bat

---   
## Question 3 (fine-tune on MNLI)
In this question you will use your feature_extractor from question 2
to fine-tune on MNLI.

(From the website):
The Multi-Genre Natural Language Inference (MultiNLI) corpus is a crowd-sourced collection of 433k sentence pairs annotated with textual entailment information. The corpus is modeled on the SNLI corpus, but differs in that covers a range of genres of spoken and written text, and supports a distinctive cross-genre generalization evaluation. The corpus served as the basis for the shared task of the RepEval 2017 Workshop at EMNLP in Copenhagen.

MNLI has 3 genres (3 classes).
The goal of this question is to maximize the test accuracy in MNLI. 

### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

In [0]:
from torchtext.datasets import MultiNLI

def init_mnli_dataset():
    """
    Fill in the details
    """
    mnli_val = None # TODO
    mnli_train = None # TODO
    mnli_test = None # TODO
    
    return mnli_train, mnli_val, mnli_test

### Part B
Here we again design a model for finetuning. Use the output of your feature-extractor as the input to this model. This should be a powerful classifier (up to you).

In [0]:
def init_finetune_model():
    
    # TODO FILL IN THE DETAILS
    fine_tune_model = ...
    
    return fine_tune_model

### Part C
Use the feature_extractor and your fine_tune_model to fine_tune MNLI

In [0]:
def fine_tune_mnli(feature_extractor, fine_tune_model, mnli_train, mnli_val):
    # YOUR CODE HERE
    pass

### Part D
Evaluate the test accuracy

In [0]:
def calculate_mnli_test_accuracy(feature_extractor, fine_tune_model, mnli_test):
    
    # YOUR CODE HERE...
    
    return test_ppl

### Let's grade your results

In [0]:
def grade_mnli():
    # load data
    mnli_train, mnli_val, mnli_test = init_mnli_dataset()

    # no need to load feature extractor because it is fine-tuned
    feature_extractor = feature_extractor

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    
    # finetune
    fine_tune_mnli(feature_extractor, fine_tune_model, mnli_train, mnli_val)

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, wikitext_test)

    # the real threshold will be released by Oct 11 
    assert test_ppl > 0.00, 'ummm... your accuracy is too low...'
    
grade_mnli()

---  
### Question 4 (BERT)

A major direction in research came from a model called BERT, released last year.  

In this question you'll use BERT as your feature_extractor instead of the model you
designed yourself.

To get BERT, head on over to (https://github.com/huggingface/transformers) and load your BERT model here

In [0]:
!pip install transformers

### Part A (init BERT)
In this section you need to create an instance of BERT and return if from the function

In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

def init_bert():
    
    BERT = None # ... YOUR CODE HERE
    
    return BERT

## Part B (fine-tune with BERT)

Use BERT as your feature extractor to finetune MNLI. Use a new finetune model (reset weights).

In [0]:
def fine_tune_mnli_BERT(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val):
    # YOUR CODE HERE
    pass

## Part C
Evaluate how well we did

In [0]:
def calculate_mnli_test_accuracy_BERT(feature_extractor, fine_tune_model, mnli_test):
    
    # YOUR CODE HERE...
    
    return test_ppl

## Let's grade your BERT results!

In [0]:
def grade_mnli_BERT():
    BERT_feature_extractor = init_bert()
    
    # load data
    mnli_train, mnli_val, mnli_test = init_mnli_dataset()

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    
    # finetune
    fine_tune_mnli(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val)

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, wikitext_test)
    
    # the real threshold will be released by Oct 11 
    assert test_ppl > 0.0, 'ummm... your accuracy is too low...'
    
grade_mnli_BERT()